In [55]:
import time, math, torch, imgaug
from sklearn import metrics
from torch.utils.data import DataLoader
from torch import optim
from torch import nn, randn, exp, sum
import warnings
warnings.filterwarnings('ignore')

In [56]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda


In [57]:
"""读取train data and label"""
import os
import cv2
import numpy as np
from torch.utils.data import Dataset

class LoadPoseData(Dataset):

    def __init__(self, thePath, is_train = True):
        super(LoadPoseData, self).__init__()
        self.dataset = []

        sub_dir = "train" if is_train else "valid"

        for tag in os.listdir(f"{thePath}/{sub_dir}"):
            file_dir = f"{thePath}/{sub_dir}/{tag}"
            for img_file in os.listdir(file_dir):
                img_path = f"{file_dir}/{img_file}"
                if tag == 'fall':
                    self.dataset.append((img_path, 0))
                else:
                    self.dataset.append((img_path, 1))

    def __len__(self):

        return len(self.dataset)

    def __getitem__(self, item):
        data = self.dataset[item]

        img = cv2.imread(data[0],cv2.IMREAD_GRAYSCALE) #以灰度图形式读数据
        
        img = img.reshape(-1) # (128,128)
        img = img/255
        
        tag_one_hot = np.zeros(2)
        tag_one_hot[int(data[1])] = 1

        return np.float32(img),np.float32(tag_one_hot)

In [58]:
import os
import cv2
import numpy as np
import random
from torch.utils.data import Dataset
from imgaug import augmenters as iaa

class LoadPoseDataEnhance(Dataset):
    def __init__(self, thePath, is_train = True, augment_minority = True, argument_per = 0.2):
        super(LoadPoseDataEnhance, self).__init__()
        self.dataset = []
        self.augment_minority = augment_minority  # 是否增强少数类
        self.minority_class = 0  # 少数类标签（'fall'）
        self.argument_per = argument_per
        
        sub_dir = "train" if is_train else "valid"

        # 统计各类样本数量
        class_counts = {0: 0, 1: 0}
        for tag in os.listdir(f"{thePath}/{sub_dir}"):
            file_dir = f"{thePath}/{sub_dir}/{tag}"
            for img_file in os.listdir(file_dir):
                img_path = f"{file_dir}/{img_file}"
                label = 0 if tag == 'fall' else 1
                self.dataset.append((img_path, label))
                class_counts[label] += 1

        # 如果需要增强少数类且训练集
        if is_train and augment_minority:
            minority_count = class_counts[self.minority_class]
            majority_count = class_counts[1 - self.minority_class]
            needed_augmentations = int(self.argument_per * majority_count) - minority_count  # 仅补充到40%

            # 从少数类样本中随机选择需要增强的样本
            minority_samples = [x for x in self.dataset if x[1] == self.minority_class]
            augment_samples = random.choices(minority_samples, k=needed_augmentations)

            # 对选中的样本进行增强并添加到数据集
            for sample in augment_samples:
                img_path, label = sample
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                augmented_img = self._augment_image(img)  # 数据增强
                self.dataset.append((augmented_img, label))  # 存储增强后的图像（非路径）

    def _augment_image(self, img):
        """对图像应用随机增强"""

        aug = iaa.Sequential([
            iaa.Affine(
                rotate=(-10, 10),       # 减小旋转范围
                scale=(0.9, 1.1),       # 轻微缩放
                translate_px=(-5, 5)    # 小幅平移
            ),
            iaa.Crop(percent=(0, 0.05)) # 最小化裁剪
            ], random_order = True)

        augmented_img = aug(image=img)
        return augmented_img

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, item):
        data = self.dataset[item]
        
        # 如果存储的是增强后的图像（直接是numpy数组）
        if isinstance(data[0], np.ndarray):
            img = data[0]
        else:
            img = cv2.imread(data[0], cv2.IMREAD_GRAYSCALE)
        
        img = img.reshape(-1)  # 展平 (128,128) -> (16384,)
        img = img / 255.0  # 归一化
        
        tag_one_hot = np.zeros(2)
        tag_one_hot[int(data[1])] = 1
        
        return np.float32(img), np.float32(tag_one_hot)

In [59]:
from torch import nn
class CNN(nn.Module):
    def __init__(self):
        # 调用父类（nn.Module）的构造函数，确保模型继承并初始化
        super().__init__()
        self.sequential = nn.Sequential(
            nn.Linear(16384, 100),
            nn.ReLU(),
            nn.Linear(100, 2),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        return self.sequential(x)

    
cnn = CNN().to(DEVICE)
print(cnn)


CNN(
  (sequential): Sequential(
    (0): Linear(in_features=16384, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=2, bias=True)
    (3): Softmax(dim=1)
  )
)


In [60]:
weight_path = "/home/bhennelly/Documents/QIN/thesis_project/pytorch-openpose-direct-v2/weights"

checkpoint_name = 'cnn_best_checkpoint.pt'

cnn.load_state_dict(torch.load(os.path.join(weight_path, checkpoint_name)))
cnn.to(DEVICE)

#加强版梯度下降法,SGD 普通梯度下降法
optimizer = optim.Adam(cnn.parameters())

In [61]:
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCH = 100000

In [62]:
class EarlyStopping_by_loss:
    def __init__(self, is_save =False, patience=300, verbose=True, delta=0, path=''):
        self.patience = patience  # 容忍多少轮验证集不提升
        self.verbose = verbose    # 是否打印信息
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.inf
        self.delta = delta        # 最小改善幅度
        self.path = path          # 保存模型路径
        self.is_save = is_save

    def __call__(self, val_loss, model, epoch, optimizer):
        score = -val_loss  # 因为我们希望 val_loss 越小越好

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model, epoch, optimizer)

        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True

        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model, epoch, optimizer)
            self.counter = 0

    def save_checkpoint(self, val_loss, model, epoch, optimizer):
        '''保存当前最优模型'''
        if self.verbose:
            print(f"Valid decreased ({self.val_loss_min:.6f} → {val_loss:.6f}).  Saving model ...")

        # （1）保存完整训练状态（用于 resume training）
        if self.is_save:
            # torch.save({
            #     'epoch': epoch,
            #     'model_state_dict': model.state_dict(),
            #     'optimizer_state_dict': optimizer.state_dict(),
            # }, self.path)
            torch.save(model.state_dict(), self.path)

        self.val_loss_min = val_loss

In [63]:
class EarlyStopping_by_f1:
    def __init__(self, is_save = False, patience=100, verbose=True, delta=0, path=''):
        self.patience = patience  # 容忍多少轮验证集不提升
        self.verbose = verbose    # 是否打印信息
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_f1_score_max = 0.0
        self.delta = delta        # 最小改善幅度
        self.path = path          # 保存模型路径
        self.is_save = is_save

    def __call__(self, val_f1_score, model, epoch, optimizer):
        score = val_f1_score  

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_f1_score, model, epoch, optimizer)

        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True

        else:
            self.best_score = score
            self.save_checkpoint(val_f1_score, model, epoch, optimizer)
            self.counter = 0

    def save_checkpoint(self, val_f1_score, model, epoch, optimizer):

        if self.verbose:
            print(f"@{epoch} valid increased from ({self.val_f1_score_max:.6f} → {val_f1_score:.6f}).  Saving model ...")
        
        # 保存当前训练情况下的最佳checkpoint，后续可以继续训练
        if self.is_save:
            # torch.save({
            #     'epoch': epoch,
            #     'model_state_dict': model.state_dict(),
            #     'optimizer_state_dict': optimizer.state_dict(),
            # }, self.path)
            torch.save(model.state_dict(), self.path)

        self.val_f1_score_max = val_f1_score

In [64]:
EARLY_STOP = 100

In [65]:
weight_save_path = '/home/bhennelly/Documents/QIN/thesis_project/pytorch-openpose-direct-v2/weights'
data_path = "/home/bhennelly/Documents/QIN/thesis_project/pytorch-openpose-direct-v2/datasets"

In [66]:
from sklearn import metrics

def train(trainLoader, validLoader, early_stopping, is_verbose=True):
    total_valid_acc = []

    for epoch in range(1, EPOCH+1):

        # ---------- TRAIN ----------
        train_sum_loss = 0
        train_sum_ap = 0
        train_sum_f1 = 0
        train_sum_acc = 0

        for train_x, train_y in trainLoader:
            train_x, train_y = train_x.to(DEVICE), train_y.to(DEVICE)
            cnn.train()

            train_pred = cnn(train_x)
            train_loss = torch.mean((train_y - train_pred)**2)

            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()

            train_sum_loss += train_loss.cpu().detach().item()

            # 转为标签
            train_y_label = torch.argmax(train_y, dim=1).cpu().numpy()
            train_pred_label = torch.argmax(train_pred, dim=1).cpu().numpy()

            train_acc = (train_y_label == train_pred_label).sum().item() / len(train_y_label)
            train_sum_acc += train_acc

            train_F1 = metrics.f1_score(train_y_label, train_pred_label)
            train_sum_f1 += train_F1

            # PR-AUC 用概率计算
            train_pred_prob = train_pred[:, 1].detach().cpu().numpy()  # 假设正类为类别1
            train_ap = metrics.average_precision_score(train_y_label, train_pred_prob)
            train_sum_ap += train_ap

        train_avg_loss = train_sum_loss / len(trainLoader)
        train_avg_f1 = train_sum_f1 / len(trainLoader)
        train_avg_acc = train_sum_acc / len(trainLoader)
        train_avg_ap = train_sum_ap / len(trainLoader)

        # ---------- VALID ----------
        valid_sum_loss = 0
        valid_sum_f1 = 0
        valid_sum_acc = 0
        valid_sum_ap = 0

        for valid_x, valid_y in validLoader:
            valid_x, valid_y = valid_x.to(DEVICE), valid_y.to(DEVICE)
            cnn.eval()
            with torch.no_grad():
                valid_pred = cnn(valid_x)
            valid_loss = torch.mean((valid_y - valid_pred)**2)
            valid_sum_loss += valid_loss.cpu().detach().item()

            valid_y_label = torch.argmax(valid_y, dim=1).cpu().numpy()
            valid_pred_label = torch.argmax(valid_pred, dim=1).cpu().numpy()

            valid_acc = (valid_y_label == valid_pred_label).sum().item() / len(valid_y_label)
            valid_sum_acc += valid_acc

            valid_F1 = metrics.f1_score(valid_y_label, valid_pred_label)
            valid_sum_f1 += valid_F1

            # PR-AUC 用概率计算
            valid_pred_prob = valid_pred[:, 1].detach().cpu().numpy()  # 假设正类为类别1
            valid_ap = metrics.average_precision_score(valid_y_label, valid_pred_prob)
            valid_sum_ap += valid_ap
            
        valid_avg_loss = valid_sum_loss / len(validLoader)
        valid_avg_f1 = valid_sum_f1 / len(validLoader)
        valid_avg_acc = valid_sum_acc / len(validLoader)
        valid_avg_ap = valid_sum_ap / len(validLoader)

        if is_verbose:
            print(f'{epoch} train | AP: {train_avg_ap:.4f}, F1: {train_avg_f1:.4f}, loss: {train_avg_loss:.4f}')
            print(f'{epoch} valid | AP: {valid_avg_ap:.4f}, F1: {valid_avg_f1:.4f}, loss: {valid_avg_loss:.4f}')

        total_valid_acc.append(valid_avg_acc)

        # 使用 PR-AUC 作为早停指标
        early_stopping(valid_avg_ap, cnn, epoch, optimizer)
        if early_stopping.early_stop:
            print("Early stopping triggered.")
            break

    return total_valid_acc


In [68]:
for argument_per in [0.2]:

    # 初始化数据集（自动增强少数类）
    train_dataset = LoadPoseDataEnhance(thePath=data_path, is_train=True, augment_minority=True, argument_per = argument_per)
    # # 检查样本数量
    # print(f"Total samples: {len(train_dataset)}")
    # print(f"Class 0 (fall) samples: {__builtins__.sum(1 for x in train_dataset.dataset if x[1] == 0)}")
    # print(f"Class 1 (non-fall) samples: {__builtins__.sum(1 for x in train_dataset.dataset if x[1] == 1)}")
    trainLoader = DataLoader(train_dataset, batch_size = TRAIN_BATCH_SIZE, shuffle = True)


    valid_dataset = LoadPoseDataEnhance(thePath=data_path, is_train=False, augment_minority=False, argument_per = argument_per)
    validLoader = DataLoader(valid_dataset, batch_size = VALID_BATCH_SIZE, shuffle=False)

    # print(len(trainLoader))
    # print(len(validLoader))

    early_stopping = EarlyStopping_by_f1(is_save=True, verbose = True, patience=EARLY_STOP, \
                                        path=os.path.join(weight_save_path, f'FLIR_best_checkpoint.pth'))

    _ = train(trainLoader, validLoader, early_stopping)


1 train | loss: 0.1783, F1: 0.8648, PR-AUC: 0.9106
1 valid | loss: 0.0463, F1: 0.9808, PR-AUC: 0.9963
@1 valid increased from (0.000000 → 0.996309).  Saving model ...
2 train | loss: 0.0914, F1: 0.9276, PR-AUC: 0.9638
2 valid | loss: 0.0859, F1: 0.9472, PR-AUC: 0.9934
EarlyStopping counter: 1 out of 100
3 train | loss: 0.0719, F1: 0.9408, PR-AUC: 0.9766
3 valid | loss: 0.1067, F1: 0.9359, PR-AUC: 0.9951
EarlyStopping counter: 2 out of 100
4 train | loss: 0.0607, F1: 0.9496, PR-AUC: 0.9818
4 valid | loss: 0.0699, F1: 0.9561, PR-AUC: 0.9950
EarlyStopping counter: 3 out of 100
5 train | loss: 0.0560, F1: 0.9525, PR-AUC: 0.9837
5 valid | loss: 0.1142, F1: 0.9244, PR-AUC: 0.9941
EarlyStopping counter: 4 out of 100
6 train | loss: 0.0533, F1: 0.9541, PR-AUC: 0.9845
6 valid | loss: 0.0899, F1: 0.9460, PR-AUC: 0.9949
EarlyStopping counter: 5 out of 100
7 train | loss: 0.0520, F1: 0.9546, PR-AUC: 0.9854
7 valid | loss: 0.0864, F1: 0.9405, PR-AUC: 0.9947
EarlyStopping counter: 6 out of 100
8 tra

In [69]:
import gc
gc.collect()

783